In [1]:
# Set the seed value for the notebook so the results are reproducible
from numpy.random import seed
seed(1)

In [2]:
# Dependencies
import numpy as np
import pandas as pd

In [3]:
import tensorflow
tensorflow.keras.__version__

'2.5.0'

# Voice Gender
Gender Recognition by Voice and Speech Analysis

This database was created to identify a voice as male or female, based upon acoustic properties of the voice and speech. The dataset consists of 3,168 recorded voice samples, collected from male and female speakers. The voice samples are pre-processed by acoustic analysis in R using the seewave and tuneR packages, with an analyzed frequency range of 0hz-280hz (human vocal range).

## The Dataset
The following acoustic properties of each voice are measured and included within the CSV:

* meanfreq: mean frequency (in kHz)
* sd: standard deviation of frequency
* median: median frequency (in kHz)
* Q25: first quantile (in kHz)
* Q75: third quantile (in kHz)
* IQR: interquantile range (in kHz)
* skew: skewness (see note in specprop description)
* kurt: kurtosis (see note in specprop description)
* sp.ent: spectral entropy
* sfm: spectral flatness
* mode: mode frequency
* centroid: frequency centroid (see specprop)
* peakf: peak frequency (frequency with highest energy)
* meanfun: average of fundamental frequency measured across acoustic signal
* minfun: minimum fundamental frequency measured across acoustic signal
* maxfun: maximum fundamental frequency measured across acoustic signal
* meandom: average of dominant frequency measured across acoustic signal
* mindom: minimum of dominant frequency measured across acoustic signal
* maxdom: maximum of dominant frequency measured across acoustic signal
* dfrange: range of dominant frequency measured across acoustic signal
* modindx: modulation index. Calculated as the accumulated absolute difference between adjacent measurements of fundamental frequencies divided by the frequency range
* label: male or female

In [4]:
voice = pd.read_csv('../Resources/voice.csv')
voice.head()

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,...,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000,male
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,...,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632,male
2,0.077316,0.083829,0.036718,0.008701,0.131908,0.123207,30.757155,1024.927705,0.846389,0.478905,...,0.077316,0.098706,0.015656,0.271186,0.007990,0.007812,0.015625,0.007812,0.046512,male
3,0.151228,0.072111,0.158011,0.096582,0.207955,0.111374,1.232831,4.177296,0.963322,0.727232,...,0.151228,0.088965,0.017798,0.250000,0.201497,0.007812,0.562500,0.554688,0.247119,male
4,0.135120,0.079146,0.124656,0.078720,0.206045,0.127325,1.101174,4.333713,0.971955,0.783568,...,0.135120,0.106398,0.016931,0.266667,0.712812,0.007812,5.484375,5.476562,0.208274,male


## Data Pre-Processing

In [5]:
X = voice.drop("label", axis=1)
y = voice["label"]
print(X.shape, y.shape)

(3168, 20) (3168,)


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)

In [8]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)

encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [10]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

# Create a Deep Learning Model

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [13]:
# Create model and add layers
model = Sequential()
model.add(Dense(units = 100, activation = 'relu', input_dim = 20))
model.add(Dense(units = 100, activation = 'relu'))

model.add(Dense(units = 2, activation = 'softmax'))#Male or Female voice

In [14]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [15]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               2100      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 202       
Total params: 12,402
Trainable params: 12,402
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.fit(X_train_scaled, y_train_categorical, epochs = 100, shuffle = True, verbose = 2)

Epoch 1/100
75/75 - 0s - loss: 0.4951 - accuracy: 0.7942
Epoch 2/100
75/75 - 0s - loss: 0.2194 - accuracy: 0.9369
Epoch 3/100
75/75 - 0s - loss: 0.1253 - accuracy: 0.9613
Epoch 4/100
75/75 - 0s - loss: 0.0976 - accuracy: 0.9718
Epoch 5/100
75/75 - 0s - loss: 0.0987 - accuracy: 0.9680
Epoch 6/100
75/75 - 0s - loss: 0.0866 - accuracy: 0.9731
Epoch 7/100
75/75 - 0s - loss: 0.0884 - accuracy: 0.9705
Epoch 8/100
75/75 - 0s - loss: 0.0863 - accuracy: 0.9718
Epoch 9/100
75/75 - 0s - loss: 0.0770 - accuracy: 0.9756
Epoch 10/100
75/75 - 0s - loss: 0.0768 - accuracy: 0.9756
Epoch 11/100
75/75 - 0s - loss: 0.0830 - accuracy: 0.9722
Epoch 12/100
75/75 - 0s - loss: 0.0760 - accuracy: 0.9756
Epoch 13/100
75/75 - 0s - loss: 0.0842 - accuracy: 0.9735
Epoch 14/100
75/75 - 0s - loss: 0.0744 - accuracy: 0.9764
Epoch 15/100
75/75 - 0s - loss: 0.0725 - accuracy: 0.9769
Epoch 16/100
75/75 - 0s - loss: 0.0743 - accuracy: 0.9760
Epoch 17/100
75/75 - 0s - loss: 0.0684 - accuracy: 0.9769
Epoch 18/100
75/75 - 0s

In [17]:
model.save("Voice_model.h5")

In [18]:
from tensorflow.keras.models import load_model

In [19]:
voice_model = load_model("Voice_model.h5")

## Quantify our Trained Model

In [20]:
#After loading, evaluate the accuracy of the model

model_loss, model_accuracy = voice_model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

25/25 - 0s - loss: 0.0669 - accuracy: 0.9798
Normal Neural Network - Loss: 0.06692985445261002, Accuracy: 0.9797979593276978


## Make Predictions

In [23]:
encoded_predictions = model.predict_classes(X_test_scaled[:10])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [24]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:10])}")

Predicted classes: ['female' 'female' 'female' 'female' 'female' 'female' 'female' 'female'
 'male' 'male']
Actual Labels: ['female', 'female', 'female', 'female', 'female', 'female', 'female', 'female', 'male', 'male']
